# Performance tips for Julia

## 1. As a rule of thumb, typed functions run faster. 
Let us revisit types first. When a type is not specified, it is assumed to be `Any`. Just a quick recap on types:

In [ ]:
function _show_subtype_tree(mytype,printlevel)
    allsubtypes = subtypes(mytype)
    for cursubtype in allsubtypes
        print("\t"^printlevel)
        println("|___",cursubtype)
        printlevel += 1
        _show_subtype_tree(cursubtype,printlevel)
        printlevel -= 1
    end
end
function show_type_tree(T)
    println(T)
    _show_subtype_tree(T,0)
end
show_type_tree(Number)

In [ ]:
function square_plus_one(v::T) where T <:Number
    g = v*v
    return g+1
end

In [ ]:
v = rand()
typeof(v)

In [ ]:
@code_warntype square_plus_one(v)

In [ ]:
w = 5
typeof(w)

In [ ]:
@code_warntype square_plus_one(w)

Great! In the above two examples, we were able to predict what the output will be. This is because:
```
function square_plus_one(v::T) where T <:Number
    g = v*v         # Type(T * T) ==> T
    return g+1      # Type(T + Int)) ==> "max" (T,Int)
end

```
Note that in both calls the return type was different, once `Float64` and once `Int64`. But the function is *still type stable*.

Now let's move to something more interesting. Let's create our first type.

In [ ]:
mutable struct Cube
    length
    width
    height
end

In [ ]:
volume(c::Cube) = c.length*c.width*c.height

In [ ]:
mutable struct Cube_typed
    length::Float64
    width::Float64
    height::Float64
end
volume(c::Cube_typed) = c.length*c.width*c.height

In [ ]:
mutable struct Cube_parametric_typed{T <: Real}
    length::T
    width::T
    height::T
end
volume(c::Cube_parametric_typed) = c.length*c.width*c.height

In [ ]:
c1 = Cube(1.1,1.2,1.3)
c2 = Cube_typed(1.1,1.2,1.3)
c3 = Cube_parametric_typed(1.1,1.2,1.3)
@show volume(c1) == volume(c2) == volume(c3)

In [ ]:
using BenchmarkTools
@btime volume(c1) # not typed
@btime volume(c2) # typed float
@btime volume(c3) # typed parametric

The second function call is the fastest! Let's call `@code_warntype` and check type stability

In [ ]:
@code_warntype volume(c1)

In [ ]:
@code_warntype volume(c2)

In [ ]:
@code_warntype volume(c3)

### Conclusion: 
Types matter, when you know anything about the types of your variables, include them in your code to make it run faster

In [ ]:
function zero_or_val(x::Real)
    if x >= 0
        return x
    else
        return 0
    end
end
@code_warntype zero_or_val(0.2)

In [ ]:
function zero_or_val_stable(x::Real)
    T = promote_type(typeof(x),Int)
    if x >= 0
        return T(x)
    else
        return T(0)
    end
end
@code_warntype zero_or_val_stable(0.2)

### Conclusion
You can avoid type instable code by using the `promote_type` function which returns the highest of the two types passed.

Let us say we want to play the following game, I give you a vector of numbers. And you want to accumulate the sum as follows. For each number in the vector, you toss a coin (`rand()`), if it is heads (`>=0.5`), you add `1`. Otherwise, you add the number itself.

In [ ]:
function flipcoin_then_add(v::Vector{T}) where T <: Real
    s = 0
    for vi in v
        r = rand()
        if r >=0.5
            s += 1
        else
            s += vi
        end
    end
end

function flipcoin_then_add_typed(v::Vector{T}) where T <: Real
    s = zero(T)
    for vi in v
        r = rand()
        if r >=0.5
            s += one(T)
        else
            s += vi
        end
    end
end
myvec = rand(1000)
@show flipcoin_then_add(myvec) == flipcoin_then_add_typed(myvec)

In [ ]:
@btime flipcoin_then_add(rand(1000))
@btime flipcoin_then_add_typed(rand(1000))

### Conclusion
Think about the variables you are declaring. Do you know their types? If so, specify the type somehow.

## 2. As a rule of thumb, **functions with preallocated memory run faster**

A classic example here is to build an array with pre-allocated memory versus pushing to it. Let's try to build Fibonacci using both approaches

In [ ]:
function build_fibonacci_preallocate(n::Int)
    @assert n >= 2
    v = zeros(Int64,n)
    v[1] = 1
    v[2] = 1
    for i = 3:n
        v[i] = v[i-1] + v[i-2]
    end
    return v
end

In [ ]:
function build_fibonacci_no_allocation(n::Int)
    @assert n >= 2
    v = Vector{Int64}()
    push!(v,1)
    push!(v,1)
    for i = 3:n
        push!(v,v[i-1]+v[i-2])
    end
    return v
end

In [ ]:
@show isequal(build_fibonacci_preallocate(10),build_fibonacci_no_allocation(10))

In [ ]:
n = 100
@btime build_fibonacci_no_allocation(n);
@btime build_fibonacci_preallocate(n);

### Conclusion
Whenever possible, preallocate memory.

It's also important to understand **how memory is organized in Julia**. Let's say, for _some reason_ you want to access all the elements of a matrix once. For the sake of this experiment, let's say we want to write `matrix_sum(A)` where A is a matrix

In [ ]:
# Create a random matrix A of size m-by-n
m = 10000
n = 10000
A = rand(m,n)
;

In [ ]:
function matrix_sum_rows(A::Matrix)
    m,n = size(A)
    mysum = 0
    for i = 1:m # fix a row
        for j = 1:n # loop over cols
            mysum += A[i,j]
        end
    end
    return mysum
end

In [ ]:
function matrix_sum_cols(A::Matrix)
    m,n = size(A)
    mysum = 0
    for j = 1:n # fix a column
        for i = 1:m # loop over rows
            mysum += A[i,j]
        end
    end
    return mysum
end

In [ ]:
function matrix_sum_index(A::Matrix)
    m,n = size(A)
    mysum = 0
    for i = 1:m*n
        mysum += A[i]
    end
    return mysum
end

In [ ]:
@show matrix_sum_cols(A) ≈ matrix_sum_rows(A) ≈ matrix_sum_index(A)

In [ ]:
@btime matrix_sum_rows(A)
@btime matrix_sum_cols(A)
@btime matrix_sum_index(A)

## Conclusion 
Matrices are organized column-wise in memory. It's better to access them one column at a time. Consider understanding how your data is organized in memory when you want to access it.

Memory recycling is when you use a chunk of memory you no longer need for another purpose

Let's take this example, you have a vector b and a vector h where b[i] is the base length of triangle i and h[i] is the height length. The experiment is to find the hypotenuse value of all triangles

In [ ]:
b = rand(1000)*10
h = rand(1000)*10
function find_hypotenuse(b::Vector{T},h::Vector{T}) where T <: Real
    return sqrt.(b.^2+h.^2)
end

In [ ]:
# Let's time it
@btime find_hypotenuse(b,h);

In [ ]:
function find_hypotenuse_optimized(b::Vector{T},h::Vector{T}) where T <: Real
    accum_vec = similar(b)
    for i = 1:length(b)
        accum_vec[i] = b[i]^2
        accum_vec[i] = accum_vec[i] + h[i]^2 # here, we used the same space in memory to hold the sum
        accum_vec[i] = sqrt(accum_vec[i]) # same thing here, to hold the sqrt
        # or:
        # accum_vec[i] = sqrt(b[i]^2+h[i]^2)
    end
    return accum_vec
end
@btime find_hypotenuse_optimized(b,h);

## Conclusion:
Whenever you can reuse memory, reuse it. 

## Bonus conclusion:
Vectorized operations are not necessarily faster.

One other form of memory recycling is in place operations

In [ ]:
function function_inplace!(v::Vector{T},myfn::Function) where T
    for i = 1:length(v)
        v[i] = myfn(v[i])
    end
    v
end

function function_not_inplace(v::Vector{T},myfn::Function) where T
    w = zeros(eltype(v),length(v))
    for i = 1:length(v)
        w[i] = myfn(v[i])
    end
    w
end

In [ ]:
v = rand(100)
@btime function_inplace!(v,x->x^2);
@btime function_not_inplace(v,x->x^2);

## Conclusion:
In-place operations are much cheaper, use them if you don't need the original data

In [ ]:
@btime v.^2;

### What are iterators and why do we care about them?
* We create iterator objects when we don't want to store/create all the elements in an array at once. 
* A quick example is a Fibonacci sequence: say you want to use the Fibonacci sequence numbers for a simple purpose but you don't necessarily care about storing all of them. You would want something like this:
``` 
fib_iterator = fib(n)
for i in fib_iterator
    #do something
end
```
In the above iteration, you are not computing and storing all the fibonacci sequence numbers. Instead, we are just creating them on the fly
* A lot of types in Julia are iteratable by default. A simple example is an array of numbers.
```
for i in rand(10)
    #do something
end
```
`rand(10)` returns a vector, and a vector is iteratable

In [ ]:
struct fib_iterator
    n::Int
end

function Base.iterate(f::fib_iterator,state=(0,0,1))
    prev1,prev2,stepid = state
    # state the ending conditions first
    if stepid == 1
        return (1,(0,1,2))
    end
    if f.n < stepid
        return nothing
    end
    # else
    y = prev1+prev2
    stepid += 1
    return (y,(prev2,y,stepid))
end

function myfib(n)
    v = zeros(Int,n+1)
    v[1] = 1
    v[2] = 1
    for i = 3:n+1
        v[i] = v[i-1] + v[i-2]
    end
    return v
end

In [ ]:
function test_iterator(n)
    f = fib_iterator(n)
    s = 0
    for i in f
        s += i
    end
end
function test_allocate(n)
    s = 0
    for i in myfib(n)
        s += i
    end
end
    
@btime test_iterator(10);
@btime test_allocate(10);

## Conclusion: 
Iterators are a powerful tool, use them when you don't need to store the values.

Always think about memory... Do you really need `A[row_ids,col_ids]`

In [ ]:
using SparseArrays
using LinearAlgebra
A = sprand(500,500,0.1)
function set_sum(A,rowids,colids)
    s = sum(A[rowids,colids])
end
function set_sum_view(A,rowids,colids)
    s = sum(view(A,rowids,colids))
end

In [ ]:
using Random
@btime set_sum(A,randperm(10), randperm(10))
@btime set_sum_view(A,randperm(10), randperm(10))

## Conclusion:
You can use views if you want to apply a function on a subset of elements.

One more idea to make your code faster: Parallelize it! But that is for a whole separat workshop (later today!)

## 3. There are many tools in Julia that helps you write faster code


### @profile

In [ ]:
# quick demo on REPL

In [ ]:
function myfunc()
    A = rand(200, 200)
    sum(A)
end

In [ ]:
using Profile

In [ ]:
@profile myfunc()

In [ ]:
Profile.print()

If you get a really long output specially when you are not expecting it, that is because Profile adds to a buffer. Try:
```
Profile.clear()
Profile.init()
```

### @inbounds

In [ ]:
?@inbounds

In [ ]:
# Let us say we want to find the sum of all elements in a vector

In [ ]:
function new_sum(myvec::Vector{Int})
    s = 0
    for i = 1:length(myvec)
        s += myvec[i]
    end
    return s
end

function new_sum_inbounds(myvec::Vector{Int})
    s = 0
    @inbounds for i = 1:length(myvec)
        s += myvec[i]
    end
    return s
end

In [ ]:
myvec = collect(1:1000000)
@btime new_sum(myvec)
@btime new_sum_inbounds(myvec)

In [ ]:
@show isequal(new_sum(myvec),new_sum_inbounds(myvec))

In [ ]:
# Be careful though!
function new_sum_WRONG(myvec::Vector{Int})
    s = 0
    for i = 1:length(myvec)+1
        s += myvec[i]
    end
    return s
end

function new_sum_inbounds_WRONG(myvec::Vector{Int})
    s = 0
    @inbounds for i = 1:length(myvec)+1
        s += myvec[i]
    end
    return s
end

myvec = collect(1:1000000);

In [ ]:
@btime new_sum_WRONG(myvec)

In [ ]:
@btime new_sum_inbounds_WRONG(myvec) # this actually exectued!

### @code_XXX
One cool thing about Julia is that it allows you to see the different stages of the code before all the way to native code! Let's look at these macros that allow you to achieve that

In [ ]:
# @code_llvm 
# @code_lowered 
# @code_native 
# @code_typed 
# @code_warntype

function flipcoin(randval::Float64)
    if randval<0.5
        return "H"
    else
        return "T"
    end
end

In [ ]:
@code_lowered flipcoin(rand()) # syntax tree

In [ ]:
@code_warntype flipcoin(rand()) # try @code_typed

In [ ]:
@code_llvm flipcoin(rand()) # this and code_warntype are probably the most relevant

In [ ]:
@code_native flipcoin(rand())